# Imports

In [ ]:
# general purpose
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import heapq
from tqdm.autonotebook import tqdm
from scipy import stats
import cv2
from collections import Counter
import warnings

# metrics and model selection
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error

# deep learning - general
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim import SGD, AdamW
from torch.optim.lr_scheduler import StepLR

# deep learning - vision utils & pretrained models
from torchvision.io import read_image, ImageReadMode
from torchvision.transforms.v2 import (
    RandomHorizontalFlip, TrivialAugmentWide, 
    Compose, Resize, ConvertDtype
)
from torchvision.transforms.functional import to_pil_image
from torchvision.models import (
    EfficientNet_V2_M_Weights, efficientnet_v2_m,
    ResNet50_Weights, resnet50
)


# customary cuda detection
device = "cuda" if torch.cuda.is_available else "cpu"
device

# Utils

In [ ]:
ROOT_PATH = Path("../input/how-much-would-you-pay-for-a-fake-cat/")
IMG_PATH = ROOT_PATH / "images" / "images"

In [ ]:
# faster than via numpy's argpartition
def topk(arraylike, k=10):
    return np.sort(heapq.nlargest(k,arraylike))

In [ ]:
# display images from tensors
def show(imgs, size=None):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False, figsize=size)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

In [ ]:
# read images - possibly all of them
def read_img(img_id):
    img_path = IMG_PATH / (img_id + ".png")
    return read_image(str(img_path),mode=ImageReadMode.RGB)
    
def read_all_imgs(img_ids, out_sz=(480,480)):
    preprocess = Compose([
            Resize(out_sz, antialias=True),
            ConvertDtype(torch.float)
    ])
    return [preprocess(read_img(img_id)) for img_id in tqdm(img_ids)]

In [ ]:
# training, evaluation and prediction utility functions
def train_one_epoch(model, optimizer, criterion, train_dataloader, device, print_frequency=None, scheduler=None):
    model.train()
    for idx, (imgs, targets) in enumerate(train_dataloader):
        optimizer.zero_grad()
        
        imgs, targets = imgs.to(device), targets.to(device)
        preds = model(imgs)
        
        # fix (n_pred, 1)-shaped predictions
        if len(preds.shape)>1:
            preds = preds.squeeze(-1)
        
        loss = criterion(targets, preds)
        
        if print_frequency is not None and (idx+1)%print_frequency == 0:
            print(f"Loss at iter {idx+1: >5}: {loss.cpu().detach().item(): .4f}")
        loss.backward()
        
        optimizer.step()
        
        if scheduler is not None:
            scheduler.step()
            
def evaluate(model, test_dataloader, device, clip=False):
    model.eval()
    
    residuals = torch.empty((len(test_dataloader)), device=device)
    preds = torch.empty((len(test_dataloader)), device=device)
    with torch.no_grad():
        for idx, (img, target) in enumerate(test_dataloader):
            output = model(img.to(device)).squeeze()
            preds[idx] = output
            residuals[idx] = preds[idx] - target.to(device)
        
        # MAE
        mae = residuals.abs().mean(0).cpu().detach().item()
        preds = preds.cpu().detach()
    
    return mae, preds           

def predict(model, test_dataloader, device, use_tqdm=True):
    model.eval()
    preds = torch.empty((len(test_dataloader)), device=device)
    with torch.no_grad():
        iterator = enumerate(test_dl)
        if use_tqdm:
            iterator = tqdm(iterator, total=len(test_dl))
        for idx, img in iterator:
            preds[idx] = model(img.to(device)).squeeze()
    return preds.cpu().detach().numpy()

In [ ]:
# submitting
def submit(preds, out_name):
    submission = test_df[["id"]].copy()
    submission["price"] = preds
    submission.to_csv(out_name+".csv", index=False)

# Loading data

In [ ]:
train_df = pd.read_csv(ROOT_PATH / "train.csv")
test_df = pd.read_csv(ROOT_PATH / "test.csv")

# Quick glance at data

In [ ]:
# peek at training data
train_df.head()

For each observation we have:

* **id**: the image id
* **price**: how much ETH it costs. This is the target variable.
* **speed**: the cooldown time for the NFT image.
* **image**: the (broken @ Kaggle FS) path to the image. 

Note that [as specified in the competition](https://www.kaggle.com/competitions/how-much-would-you-pay-for-a-fake-cat/data), the image path always has the "image/"+ id + ".png" format, hence the image path is a redundant feature.

In [ ]:
# drop the "image" feature as it's of no use
for df in (train_df, test_df):
    df.drop(columns=["image"], inplace=True)

In [ ]:
# types + any missing values?
train_df.info()

## Price



In [ ]:
# weird that it's a string, is there some weird character around?
for x in train_df["price"].unique():
    try:
        x = float(x)
    except ValueError as e:
        print(x,"---", str(e))

In [ ]:
# deleting the commas as they make no difference + converting to float
train_df["price"] = train_df["price"].str.replace(",","").astype(float)

In [ ]:
# general stats for the target variable
train_df["price"].describe()

In [ ]:
# coarse-grained look -- a box plot
sns.boxplot(train_df, y="price")
plt.show()

We can see two things:
* The data is highly skewed
* There seem to be *a lot* of values outside the Q1-Q3 range, and even then we have a clear outlier on top.

In [ ]:
topk(train_df["price"], k=10)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

# ECDF on log scale
ax = sns.ecdfplot(train_df,x="price",log_scale=True,ax=ax, color="blue")

# we want a finer granularity than default 0.2 -- y ticks every 0.1
ax.set_yticks(np.linspace(0,1,11))
ax.grid()

# show Q1 and Q3
for q in (0.25,0.75):
    ax.axhline(q, xmin=0, xmax=1, linestyle="dashed", color="orange")

plt.show()

1. Almost 40% of the samples cost at most 1 ETH
2. Over 90% of the samples cost below 100 ETH, the fraction raises to ~99% for 1000 ETH
3. From the top-k we know all top-5 prices are outliers

## Speed

In [ ]:
# how many unique values do we find out of the 2k samples?
train_df["speed"].nunique()

In [ ]:
# is there any unseen value at test?
set(test_df["speed"])-set(train_df["speed"])

So only 14 unique values and no new ones at test, we can look at the training set ones and encode such values accordingly!

In [ ]:
pd.value_counts(train_df["speed"])

This is an inherently (positive) real-valued feature that measures time interval durations. As the shortest one is 1 minute, we can encode it as the number of minutes.

In [ ]:
# encoding as minutes
def encode_speed(speed):
    # faster than regex
    cd = speed.split("(")[1].split(")")[0]
    num, unit = int(cd[:-1]), cd[-1]
    translate={
        'm': 1,
        'h': 60,
        'd': 60*24,
        'w': 60*24*7
    }
    return num * translate[unit]

# test drive
sample = train_df["speed"][3]
sample, encode_speed(sample)

In [ ]:
# apply encoding
for df in (train_df, test_df):
    df["speed_encoded"] = df["speed"].apply(encode_speed)
    
train_df.head()

But is it a good regressor?

In [ ]:
# plot
sns.violinplot(x=train_df["speed_encoded"], y=np.log10(train_df["price"]), log_scale=True)

In [ ]:
# how well does it correlate with price?
pearson = np.corrcoef(train_df["speed_encoded"],train_df["price"])[0,1]

spearman = stats.spearmanr(train_df["speed_encoded"],train_df["price"])[0]

print(f"{pearson=:.3}\n{spearman=:.3}")

While it doesn't look like a great predictor in the continuous sense, it seems that some groups are not as sparsely distributed. We can exploit this behavior on a group-wise statistic-based baseline model.

# Baseline: predicting price using only speed group statistics

In [ ]:
def MAD(arraylike):
    return np.median((arraylike - np.median(arraylike)).abs())

def IQR(arraylike):
    x = np.quantile(arraylike, (0.25, 0.75))
    return x[1] - x[0]

def trimmed_mean(arraylike, p=0.05):
    lower, upper = np.quantile(arraylike, (p, 1-p))
    return arraylike[(arraylike >= lower) & (arraylike <= upper)].mean()
    

def get_measures(df):
    res = df.groupby("speed_encoded").agg({"price": [len, min,  max, 
                                                     np.median, MAD, IQR, 
                                                     np.mean, np.std, 
                                                     trimmed_mean, 
                                                     lambda x: trimmed_mean(x,0.1)]})
    # reset column index - not so straightforward
    res.columns = list(res.columns.get_level_values(1))[:-2]+[f"trimmed_mean_{p}" for p in ("05","1")]
    return res

train_measures = get_measures(train_df)
train_measures

In [ ]:
FOLDS = 5
splitter = StratifiedKFold(FOLDS, shuffle=True, random_state=42)
oof_predictions = train_df[["speed_encoded", "price"]].copy()
features = ["mean", "median", "IQR", "MAD", "std", "trimmed_mean_05", "trimmed_mean_1"]
for feature in features:
    oof_predictions[feature] = 0

preds = {
    "mean": lambda df: df["mean"],
    "median": lambda df: df["median"],
    "tmean_05": lambda df: df["trimmed_mean_05"],
    "tmean_1": lambda df: df["trimmed_mean_1"],
    "median-IQR": lambda df: df["median"]-df["IQR"]/2,
    "median-MAD": lambda df: df["median"]-df["MAD"],
    "tmean_05-MAD": lambda df: df["trimmed_mean_05"]-df["MAD"],
    "tmean_1-MAD": lambda df: df["trimmed_mean_1"]-df["MAD"],
}

errors = {k: np.empty(FOLDS) for k in preds}

for idx, (train_idxs, test_idxs) in enumerate(splitter.split(train_df, train_df["speed_encoded"])):
    train = train_df.loc[train_idxs]
    oof = oof_predictions.loc[test_idxs]
    
    measures = get_measures(train)[features]
    merged = (
        oof[["speed_encoded"]]
        .merge(measures,left_on="speed_encoded",right_index=True)
    )
    
    for k, f in preds.items():
        errors[k][idx] = mean_absolute_error(oof["price"], f(merged))
    
    oof_predictions.loc[test_idxs, features] = merged
    
    
oof_predictions

In [ ]:
results = pd.DataFrame(errors)
results

We can easily see that:
* Per-fold MAE has huge variability, mostly due to mentioned outliers. Even if MAE is a robust metric, with too many orders of magnitude the performance values are still dominated by said extreme samples.
* Median and 0.1-trimmed mean (and their MAD-discounted variations) strictly outperform all the others.

Because of its simplicity and its invariability with respect to any work we perform on the more extremed values (e.g. clipping them) we will go for the group-median as a baseline model.

In [ ]:
# predict with median
median_preds = (
    test_df[["id","speed_encoded"]]
    .merge(train_measures[["median"]], left_on="speed_encoded", right_index=True)
)["median"].to_numpy()

In [ ]:
submit(median_preds, "median")

# Clipping the targets

Given how the more extreme values dominate the MAE we can expect any supervised model that directly minimizes it to suffer from this effect. Two possible ways of dealing with this are clipping the targets and dropping the *toxic* samples.
As clipping preserves the size of the dataset and possibly lets us learn "high-price" features without forcing the models to only learn to predict those correctly, we will opt for this.

Thresholds of 100, 500 and 1000 were tried, with 100 maximizing the unclipped-validation-set performance.

In [ ]:
CLIP_THRESHOLD = 100

In [ ]:
# keep clipped prices
clipped_prices = train_df["price"].clip(upper=CLIP_THRESHOLD)

# Finetuning pretrained Vision models

We will now finetune models pretrained on ImageNet, which we will modify by changing their last (classifier) layer for a regression one. The full list of models can be found [here](https://pytorch.org/vision/stable/models.html#table-of-all-available-classification-weights).

* [EfficientNet V2 M](https://pytorch.org/vision/stable/models/generated/torchvision.models.efficientnet_v2_m.html#torchvision.models.EfficientNet_V2_M_Weights) and [ResNet50 (V2 weights)](https://pytorch.org/vision/stable/models/generated/torchvision.models.resnet50.html#torchvision.models.ResNet50_Weights) were considered.
* Three kinds of losses were explored: L1, [SmoothL1](https://pytorch.org/docs/stable/generated/torch.nn.SmoothL1Loss.html) and a custom huber-like Linear-Root one.
* Two kinds of activations were considered:
    * Linear: can map all real numbers, has the same gradient for any target value.
    * [Softplus](https://pytorch.org/docs/stable/generated/torch.nn.Softplus.html): maps only positive numbers (a pro) but may have a harder time learning to predict very small values due to small/vanishing gradients (this is a con).

## Sanity checks

In [ ]:
# all ids have their corresponding image path
no_img = [[
    img_id for img_id in df["id"]
    if not (IMG_PATH / (img_id+".png")).exists()
] for df in (train_df, test_df)]

no_img

In [ ]:
script false --no-raise-error
# do all images in the training set have the same size?
img_sizes = [cv2.imread(str(IMG_PATH / (img_id+".png"))).shape for img_id in train_df["id"]]
Counter(img_sizes)

* All images are available for all rows in both the training and test set
* There are multiple sizes for the images, although one accounts for the majority of them. At test time and for the models this is not a problem as they resize as part of their preprocessing pipeline, but larger than 1 batches cannot be constructed for differently sized images. Prior resizing will be needed.

## Datasets and augmentation transforms

Given how small the dataset is, we can get away with preemptively loading them in memory, making the training loop faster by removing the I/O bottleneck.

In [ ]:
class PreLoadedImageDataset(Dataset):
    def __init__(self, imgs, df=None, targets=None, transforms=None, train=True):
        assert(df is not None or targets is not None)
        self.imgs = imgs
        if train is False:
            self.target = None
        else:
            if df is not None:
                self.target = df["price"].to_numpy()
            if targets is not None:
                self.target = targets
        self.t = transforms
    
    def __len__(self):
        return len(self.imgs)
    
    def __getitem__(self, idx):
        img = self.imgs[idx]
        res = (img, self.target[idx]) if self.target is not None else img
        return self.t(res) if self.t is not None else res

In [ ]:
train_imgs = read_all_imgs(train_df["id"])
test_imgs = read_all_imgs(test_df["id"])

While data augmentation is always good, we have to pick transforms that do not take information away from the images. As we do not know whether color, shape or size matter and upside-down cats don't make sense, we will only use Random Horizontal Flipping. Even though it's only one transformation, this will efectively double our training set size.

In [ ]:
def get_transforms(p):
    return RandomHorizontalFlip(p)

In [ ]:
# clipped targets + augmentation on train
train_dataset = PreLoadedImageDataset(train_imgs, targets=clipped_prices, transforms=get_transforms(0.5))

# unclipped targets + no augmentation on valid
valid_dataset = PreLoadedImageDataset(train_imgs, df=train_df, transforms=None)

# no targets on test
test_dataset = PreLoadedImageDataset(test_imgs, df=test_df, transforms=None, train=False)

Now we split the training set in train and validation subsets by performing a random train-test split. Considering the test set has 120 samples, we keep only a 20% of the train set for validation, which still means 400 samples.

In [ ]:
VALID_SZ = 400
VALID_RNG = 42

In [ ]:
g = torch.manual_seed(VALID_RNG)
idxs = torch.randperm(len(train_df), generator=g).tolist()

train_subset = Subset(train_dataset, idxs[:-VALID_SZ])
valid_subset = Subset(valid_dataset, idxs[-VALID_SZ:])

## DataLoaders

For a more stable training and faster iterations we pick a rather sizeable batch size. Lower values were explored, with no performance gains.

In [ ]:
TRAIN_BATCH_SZ = 128
N_WORKERS = 1

In [ ]:
train_dl = DataLoader(
    train_subset, 
    batch_size=TRAIN_BATCH_SZ, 
    shuffle=True,
    num_workers=N_WORKERS
)

valid_dl = DataLoader(
    valid_subset, 
    batch_size=1, 
    shuffle=False,
    num_workers=N_WORKERS
)

test_dl = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=N_WORKERS
)

len(train_dl),len(valid_dl), len(test_dl)

## Models

In [ ]:
# base class
class BackboneRegressor(nn.Module):
    def __init__(self, extractor_features, regressor=None, use_sp=True):
        super().__init__()
        if regressor is None:
            regressor = nn.Linear(
                in_features = extractor_features,
                out_features = 1
            )
        self.preprocessor, self.backbone = self.setup(regressor)
        self.backbone.to(device)
        self.sp = nn.Softplus() if use_sp else None
    
    def setup(self, regressor):
        raise NotImplementedError("override me!")
        
    def forward(self, x):
        x = self.preprocessor(x)
        x = self.backbone(x)
        return self.sp(x) if self.sp is not None else x  

In [ ]:
# EfficientNet
class EfficientNetRegressor(BackboneRegressor):
    def __init__(self, regressor=None, use_sp=True):
        super().__init__(
            extractor_features=1280,
            regressor=regressor, 
            use_sp=use_sp
        )
    
    def setup(self, regressor):
        w = EfficientNet_V2_M_Weights.DEFAULT
        bb = efficientnet_v2_m(weights=w)
        bb.classifier = regressor
        return w.transforms(), bb
    
# ResNet
class ResNetRegressor(BackboneRegressor):
    def __init__(self, regressor=None, use_sp=True):
        super().__init__(2048,regressor, use_sp)
    
    def setup(self, regressor):
        w = ResNet50_Weights.DEFAULT
        bb = resnet50(weights=w)
        bb.fc = regressor
        return w.transforms(), bb

In [ ]:
USE_SOFTPLUS = True
REGRESSOR = None
MODEL_CLS = ResNetRegressor

In [ ]:
model = MODEL_CLS(use_sp=USE_SOFTPLUS, regressor=REGRESSOR)

## Optimizer and Scheduler


In [ ]:
params = [p for p in model.parameters() if p.requires_grad]
len(params)

In [ ]:
LR = 1e-3
MOMENTUM = 0.9
W_DECAY = 0.01
STEP_SZ = 50
LR_DECAY = 0.5

In [ ]:
# previous recipe
"""
optimizer = SGD(
    params,
    lr=LR,
    momentum=MOMENTUM,
    weight_decay=W_DECAY
)

scheduler = StepLR(
    optimizer=optimizer,
    step_size= STEP_SZ,
    gamma=LR_DECAY
)
"""

In [ ]:
# current recipe - a simpler, yet more aggressive one
optimizer = AdamW(
    params,
    lr=LR,
    weight_decay=W_DECAY
)

scheduler = None

## Loss

In [ ]:
# defining the custom Linear-SQRT loss
class RootLoss(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, yhat, y):
        residuals = torch.abs(yhat-y)
        rooted_res = 2*torch.sqrt(residuals)-1
        return torch.mean(torch.where(residuals > 1, rooted_res, residuals))

In [ ]:
# this is how it looks like - linear below 1, sqrt after
yhat = torch.linspace(start=1e-3,end=5,steps=1000)
ypp = torch.zeros_like(yhat)
rl = RootLoss()

_ = [rl(yhat[idx],ypp[idx]) for idx in range(len(yhat))]
plt.plot(yhat, _)

While this loss puts even less importance on larger residuals, it didn't improve performance over SmoothL1Loss.

In [ ]:
#loss = nn.L1Loss()
#loss = RootLoss()
loss = nn.SmoothL1Loss()

## Training loop

In [ ]:
N_EPOCHS = 60

In [ ]:
# suppress pesky UserWarning from antialias value from inside the ResNet preprocessing code
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
preds = {}
mae_historic = np.empty(N_EPOCHS, dtype=float)
clipped_mae_historic = np.empty(N_EPOCHS, dtype=float)
clip_min = 13
unclip_min = 18

clipped_mae_threshold = 11 # for clipped targets
valid_clipped_prices = clipped_prices[idxs[-VALID_SZ:]]

for epoch in range(N_EPOCHS):
    # train 1 epoch
    train_one_epoch(
        train_dataloader=train_dl,
        model=model,
        optimizer=optimizer,
        scheduler=scheduler,
        criterion = loss, 
        device=device
    )
    # calculate, save and report unclipped and clipped MAE
    mae, valid_preds = evaluate(model,valid_dl,device)
    clipped_mae = mean_absolute_error(valid_clipped_prices, valid_preds)
    
    mae_historic[epoch] = mae
    clipped_mae_historic[epoch] = clipped_mae
    
    clip_min = min(clip_min, clipped_mae)
    unclip_min = min(unclip_min, mae)
    
    # if it's below a threshold also 
    if clipped_mae < clipped_mae_threshold:
        preds[epoch] = predict(model, test_dl, device, use_tqdm=False)
    
    print(
        f"Epoch {epoch+1: >2}/{N_EPOCHS}",
        f"UNCLIPPED MAE: {mae:>.3f}" + (" <--" if unclip_min == mae else "\t"), 
        f"CLIPPED MAE: {clipped_mae:>.3f}" + (" <--" if clip_min == clipped_mae else "\t"),
        sep="\t"
    )
preds[N_EPOCHS] = predict(model, test_dl, device)

In [ ]:
# which ones were saved?
preds.keys()

In [ ]:
# extremely inefficient yet numba-compilable
def rolling_mean(arraylike, lambda_, exclude_first=0, fill=None):       
    res = np.copy(arraylike) if fill is None else np.full_like(arraylike, fill)
    aux = arraylike[exclude_first]
    for idx, x in enumerate(arraylike[exclude_first:]):
        aux = aux*lambda_ + x*(1-lambda_)
        res[idx+exclude_first] = aux
    return res

In [ ]:
fig, axes = plt.subplots(figsize=(8,6), nrows=2, sharex=True)

# plot rolling mean+-std
for ax, historic, name in zip(axes, 
                              (mae_historic, clipped_mae_historic), 
                              ("Unclipped MAE", "Clipped MAE")):
    lambda_ = 0.8
    ewma = rolling_mean(historic, lambda_, 1, np.nan)
    ewmstd = np.sqrt(rolling_mean((historic-ewma)**2,lambda_, 1, 0))
    ax.fill_between(range(len(historic)),ewma+ewmstd, ewma-ewmstd,color='lightgray')
    ax.plot(ewma, color='gray')

    min_mae = np.argmin(historic)
    print("Min",name," at",min_mae)
    for k in preds.keys():
        color = 'red' if k==min_mae else 'orange'
        ax.axvline(k, color=color, linestyle='dashed')

    # plot valid MAE trajectory
    ax.plot(historic, color='blue')
    ax.set_ylabel(name)
plt.show()

In [ ]:
# selecting the min unclipped MAE model
selected_idx = 20

In [ ]:
# generate submission
submit(preds[selected_idx],"resnet"+str(selected_idx)+("sp" if USE_SOFTPLUS else ""))